In [8]:
import arcpy
import os
from dotenv import load_dotenv
from pathlib import Path
import json
import arcpy
from arcpy.da import InsertCursor
from shapely.geometry import Point
import random
import math

###################################################
########## path to geodatabase ####################
load_dotenv() # --> load in path via env variable 
raw_path:str = os.getenv('GDB_PATH')
GDB_PATH:str = r"{}".format(raw_path)
if arcpy.Exists(GDB_PATH): 
    print(f"Geodatabase exists: {GDB_PATH}")
else:
    print(f"Geodatabase DNE: {GDB_PATH}")

###################################################
#####  Spatial Refernce for Project: WGS 84  ######

SPATIAL_REF= arcpy.SpatialReference(4326) # WGS 84 (latitude/longitude)
#SPATIAL_REF = arcpy.SpatialReference(26918)


### Import the grocery store feature class ###
GROCERY_FC = "grocery_store_fc"
GROCERY_FC_PATH = os.path.join(GDB_PATH,GROCERY_FC)

### Import the unhealthy food feature class ###
UNHEALTHY_FOOD_FC = "unhealthy_food_fc"
UNHEALTHY_FOOD_FC_PATH = os.path.join(GDB_PATH,UNHEALTHY_FOOD_FC)

### grocery store buffer feature class for the 0.5 mile radius circle ###
GROCERY_BUFFER_FC = "grocery_buffer_fc"
GROCERY_BUFFER_FC_PATH = os.path.join(GDB_PATH,GROCERY_BUFFER_FC)

Geodatabase exists: C:\Users\jbfre\Documents\ArcGIS\Projects\From_Desert_2_Oasis\From_Desert_2_Oasis.gdb


In [9]:
### Verify that Grocery Store and unhealthy Food Feature classes exist ###
if arcpy.Exists(UNHEALTHY_FOOD_FC_PATH):
    print("unhealthy food feature class exists ")
if arcpy.Exists(GROCERY_FC_PATH):
    print("grocery store feature class exists ")
if arcpy.Exists(GROCERY_BUFFER_FC_PATH):
    print("grocery store BUFFER feature class exists ")

unhealthy food feature class exists 
grocery store feature class exists 
grocery store BUFFER feature class exists 


In [16]:
### Create new Oasis feature class ###
OASIS_FC = "oasis_fc"
OASIS_FC_PATH = os.path.join(GDB_PATH,OASIS_FC)

### Create new Oasis BUFFER feature  class ###
OASIS_BUFFER_FC = "oasis_buffer_fc"
OASIS_BUFFER_FC_PATH = os.path.join(GDB_PATH,OASIS_BUFFER_FC)


# Create output feature classes if they do not exist
if not arcpy.Exists(OASIS_FC_PATH):
    arcpy.CreateFeatureclass_management(GDB_PATH, OASIS_FC, "POINT")
if not arcpy.Exists(OASIS_BUFFER_FC_PATH):
    arcpy.CreateFeatureclass_management(GDB_PATH, OASIS_BUFFER_FC, "POLYGON")

In [21]:
# def check_grocery_nearby(unhealthy_food_point):
#     # Create a 0.5-mile buffer around the unhealthy food point
#     buffer_05mile = "in_memory\\0.5mile_buffer"
#     arcpy.analysis.Buffer(unhealthy_food_point, buffer_05mile, "0.5 Miles")
    
#     # Find grocery stores within the 0.5-mile buffer
#     grocery_in_buffer = "in_memory\\grocery_in_buffer"
#     arcpy.analysis.Select(GROCERY_FC_PATH, grocery_in_buffer, f"SHAPE WITHIN {buffer_05mile}")
    
#     # Check if any grocery store is within 0.5 miles of the unhealthy food point
#     for grocery in arcpy.da.SearchCursor(grocery_in_buffer, ["SHAPE"]):
#         if grocery[0].distanceTo(unhealthy_food_point) <= 0.5 * 5280:  # 0.5 miles in feet
#             return True  # Grocery store found nearby
    
#     return False  # No grocery store found nearby

# def check_grocery_nearby(unhealthy_food_point):
#     # Create a 0.5-mile buffer around the unhealthy food point
#     buffer_05mile = "in_memory\\0.5mile_buffer"
#     arcpy.analysis.Buffer(unhealthy_food_point, buffer_05mile, "0.5 Miles")
    
#     # Use a SearchCursor to find grocery stores within the 0.5-mile buffer
#     grocery_in_buffer = "in_memory\\grocery_in_buffer"
#     arcpy.management.SelectLayerByLocation(GROCERY_FC_PATH, "WITHIN", buffer_05mile)

#     # If any grocery store is found in the buffer, return True
#     if int(arcpy.GetCount_management(GROCERY_FC_PATH)[0]) > 0:
#         return True
    
#     return False  # No grocery store found within the 0.5-mile buffer

def check_grocery_nearby(unhealthy_food_point):
    # Create a 0.5-mile buffer around the unhealthy food point
    buffer_05mile = "in_memory\\0.5mile_buffer"
    arcpy.analysis.Buffer(unhealthy_food_point, buffer_05mile, "0.5 Miles")
    
    # Select grocery stores within the 0.5-mile buffer
    arcpy.management.SelectLayerByLocation("grocery_layer", "WITHIN", buffer_05mile)
    
    # Check if any grocery store is within the buffer
    if int(arcpy.GetCount_management("grocery_layer")[0]) > 0:
        return True
    
    return False  # No grocery store found within the 0.5-mile buffer


In [22]:
def add_oasis_and_buffer(unhealthy_food_point):
    # Generate random point within 0.5 miles of the unhealthy food point
    # (0.5 miles = 2640 feet)
    distance = 2640  # 0.5 miles in feet
    random_angle = random.uniform(0, 360)
    x_offset = distance * random.uniform(0, 1) * math.cos(math.radians(random_angle))
    y_offset = distance * random.uniform(0, 1) * math.sin(math.radians(random_angle))
    
    # Create the new oasis point
    oasis_point = arcpy.Point(unhealthy_food_point.X + x_offset, unhealthy_food_point.Y + y_offset)
    
    # Insert the new point into the oasis feature class
    with arcpy.da.InsertCursor(OASIS_FC_PATH, ["SHAPE"]) as cursor:
        cursor.insertRow([arcpy.PointGeometry(oasis_point)])
    
    # Create a buffer around the new oasis grocery store
    oasis_buffer = "in_memory\\oasis_buffer"
    arcpy.analysis.Buffer(arcpy.PointGeometry(oasis_point), oasis_buffer, "0.5 Miles")
    
    # Insert the new oasis buffer into the buffer feature class
    with arcpy.da.InsertCursor(OASIS_BUFFER_FC_PATH, ["SHAPE"]) as cursor:
        cursor.insertRow([arcpy.CopyFeatures_management(oasis_buffer)])


In [24]:
# Loop through all unhealthy food locations
with arcpy.da.SearchCursor(UNHEALTHY_FOOD_FC_PATH, ["SHAPE"]) as cursor:
    for row in cursor:
        unhealthy_food_point = row[0]
        
        # Create a 0.5-mile buffer around the unhealthy food point
        if not check_grocery_nearby(unhealthy_food_point):
            # If no grocery store is within 0.5 miles, add an oasis grocery store and create a buffer around it
            add_oasis_and_buffer(unhealthy_food_point)


RuntimeError: Object: Error in executing tool